In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from time import time

import matplotlib.pyplot as plt
import pandas as pd

from njit_funcs import round_dynamic
from procedures import create_binance_bot, create_binance_bot_spot, create_bybit_bot, load_exchange_key_secret
from pure_funcs import get_template_live_config, get_daily_from_income

using numba


In [6]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)

In [7]:
config = get_template_live_config()
config['market_type'] = 'futures'
config['user'] = 'binance_01'
config['exchange'], _, _ = load_exchange_key_secret(config['user'])

n_days = 30
start_time = (time() - 60 * 60 * 24 * n_days) * 1000
end_time = time() * 1000

symbols = ['XTZUSDT', 'ATOMUSDT', 'XLMUSDT', 'OMGUSDT', 'MANAUSDT', 'NEARUSDT', 'FTMUSDT', 'DOGEUSDT', 'LINKUSDT',
           'LUNAUSDT', 'MATICUSDT']

incomes = []
# binance fetches income for all symbols; bybit only one symbol at a time
for symbol in (symbols if config['exchange'] == 'bybit' else symbols[:1]):
    config['symbol'] = symbol
    try:
        await bot.session.close()
    except:
        pass
    if config['exchange'] == 'binance':
        if config['market_type'] == 'spot':
            bot = await create_binance_bot_spot(config)
        else:
            bot = await create_binance_bot(config)
    elif config['exchange'] == 'bybit':
        bot = await create_bybit_bot(config)

    await bot.update_position()
    balance = bot.position['wallet_balance']

    income = await bot.get_all_income(start_time=start_time)
    incomes += income
idf, bdf = get_daily_from_income(sorted(incomes, key=lambda x: x['timestamp']), bot.position['wallet_balance'],
                                 start_time=start_time, end_time=end_time)


CancelledError: 

In [ ]:
print(f'current balance {round_dynamic(balance, 5)}')
print(f'abs sum {round_dynamic(idf.income.sum(), 4)} {idf.iloc[-1]["token"]}')
print(f'abs adg {round_dynamic(idf.income.sum() / n_days, 4)} {idf.iloc[-1]["token"]}')
print(f'pct sum {((balance + idf.income.sum()) / balance) - 1:.5f}')
print(f'pct adg {((balance + idf.income.sum()) / balance) ** (1 / n_days) - 1:.5f}')

idf.income.cumsum().plot()

In [ ]:
idf.tail(20)

In [ ]:
bdf

In [ ]:
bdf.gain.plot()

In [ ]:
# check income last x days
x = 3
since = (time() - 60 * 60 * 24 * x) * 1000
idf[idf.timestamp > since].groupby('symbol').income.sum().sort_values()